<a href="https://colab.research.google.com/github/jwb4335/nlp/blob/master/hw2_part1/hw2part1_johnbarry_20200120.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework 2 part 1
## CS 590.07
## John Barry





## Load in simlex999 data from github

In [0]:
import pandas as pd
from random import randint



In [0]:
url = 'https://raw.githubusercontent.com/jwb4335/nlp/master/hw2_part1/Sim_Lex999.txt'
simlex = pd.read_csv(url,sep = ':')
simlex.columns = ['word1','word2','SimLex999','POS1','POS2']



```
# This is formatted as code
```

Let's get three random rows

In [3]:
randints = [randint(0,len(simlex)), randint(0,len(simlex)),randint(0,len(simlex))]

simlex_rand = simlex.iloc[randints,:]
simlex_rand



,word1,word2,SimLex999,POS1,POS2
178,street,alley,5.48,n,n
498,car,carriage,5.13,n,n
971,accept,deliver,1.58,v,v


## **Universal sentence encoder** 





In [0]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
!pip3 install --quiet seaborn

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [7]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Compute a representation for each message, showing various lengths supported.
word = "breakfast"
messages = [word]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: breakfast
Embedding size: 512
Embedding: [-0.016404513269662857, -0.04483848437666893, -0.0037806909531354904, ...]



In [8]:
message_embedding_snippet

'-0.016404513269662857, -0.04483848437666893, -0.0037806909531354904'

In [0]:
word = "breakfast"
messages = [word]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

In [10]:
message_embeddings

array([[-1.64045133e-02, -4.48384844e-02, -3.78069095e-03,
         4.68277894e-02, -7.71008851e-03, -5.05571701e-02,
        -6.78127445e-03, -3.27876322e-02, -6.28904551e-02,
        -4.78958301e-02,  5.76800331e-02, -5.30807972e-02,
         2.26386786e-02,  2.00913250e-02, -1.91726331e-02,
         1.45138241e-02, -9.16321948e-03, -3.39176925e-03,
        -2.80498969e-03, -3.79767418e-02,  4.18685153e-02,
        -6.24692850e-02, -2.87507810e-02, -3.40368375e-02,
        -3.47199216e-02, -2.53357105e-02, -4.59449776e-02,
        -7.09898174e-02, -6.53066039e-02,  3.80705930e-02,
        -3.92757617e-02, -2.31163409e-02, -3.94491926e-02,
         1.65286213e-02,  3.73129509e-02, -5.67618310e-02,
        -3.04891113e-02,  5.97833432e-02, -2.56873239e-02,
        -6.43153442e-04,  5.93997464e-02,  5.53347953e-02,
         2.13192087e-02, -5.59708215e-02,  4.60403711e-02,
         2.85874866e-02,  3.25253457e-02, -7.22681209e-02,
         5.07617891e-02, -1.03512416e-02, -5.29682003e-0

# CBOW or word2index

Download the data

In [11]:
!wget "https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/reviews.txt"
!cat reviews.txt | head

--2020-01-25 21:58:34--  https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33678267 (32M) [text/plain]
Saving to: ‘reviews.txt’

reviews.txt         100%[===================>]  32.12M   130MB/s    in 0.2s    

2020-01-25 21:58:35 (130 MB/s) - ‘reviews.txt’ saved [33678267/33678267]

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the who

In [12]:
!wget "https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/labels.txt"
!cat labels.txt | head -15

--2020-01-25 21:58:38--  https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225000 (220K) [text/plain]
Saving to: ‘labels.txt’

labels.txt          100%[===================>] 219.73K  --.-KB/s    in 0.02s   

2020-01-25 21:58:38 (12.9 MB/s) - ‘labels.txt’ saved [225000/225000]

positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive


Predicting Movie Reviews

In [0]:
import sys

f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

f = open('labels.txt')
raw_labels = f.readlines()
f.close()

tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

In [14]:
vocab[11]

'impacting'

In [15]:
vocab

['waylaid',
 'frats',
 'skate',
 'mnard',
 'preempt',
 'creepiness',
 'onyulo',
 'equiptment',
 'intensional',
 'lillete',
 'postmortem',
 'impacting',
 'focused',
 'bey',
 'belmondo',
 'effeminately',
 'kolchak',
 'thorstein',
 'irrelevant',
 'darkangel',
 'gnashingly',
 'brazzi',
 'jacy',
 'stefaniuk',
 'certified',
 'correctly',
 'davidtz',
 'renovation',
 'clausen',
 'sloan',
 'amlie',
 'smrgsbord',
 'wail',
 'unilluminated',
 'contacted',
 'interviewer',
 'crookedness',
 'liverpudlian',
 'anderson',
 'mcdougall',
 'offence',
 'cockles',
 'caetano',
 'seawall',
 'paoli',
 'abi',
 'highjinx',
 'huhuhuhuhu',
 'directionand',
 'eventully',
 'hush',
 'jeayes',
 'yewbenighted',
 'lilies',
 'urucows',
 'ooooof',
 'bihari',
 'baboon',
 'laterally',
 'primetime',
 'kabul',
 'chandeliers',
 'vernon',
 'discontented',
 'begins',
 'memorizing',
 'daninsky',
 'doink',
 'unforgettably',
 'foole',
 'odyessy',
 'tagore',
 'spanishness',
 'decks',
 'fluent',
 'floraine',
 'tehmul',
 'otvio',
 'egy

Initialize Weights

In [16]:
import numpy as np
np.random.seed(1)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

alpha, iterations = (0.01, 2)
hidden_size = 100

weights_0_1 = 0.2*np.random.random((len(vocab),hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,1)) - 0.1


print (weights_0_1)

[[-0.0165956   0.0440649  -0.09997713 ...  0.0147359  -0.09942593
   0.02342898]
 [-0.03467102  0.00541162  0.07718842 ... -0.05312758  0.02335567
   0.08980326]
 [ 0.09003522  0.01133064  0.08312127 ... -0.08407478  0.09656342
  -0.06367743]
 ...
 [-0.07656205 -0.09023205 -0.00978559 ... -0.0008865  -0.04352412
  -0.06427378]
 [-0.08444666 -0.07143386 -0.05731608 ... -0.06584719 -0.02770722
   0.09327983]
 [ 0.03695404 -0.05106593  0.02970002 ...  0.0043325   0.00801544
  -0.09468272]]


Run the Model

In [17]:


correct,total = (0,0)

for iter in range(iterations):
    
    # train on first 24,000
    for i in range(len(input_dataset)-1000):

        x,y = (input_dataset[i],target_dataset[i])
        layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0)) #embed + sigmoid
        layer_2 = sigmoid(np.dot(layer_1,weights_1_2)) # linear + softmax

        layer_2_delta = layer_2 - y # compare pred with truth
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) #backprop

        weights_0_1[x] -= layer_1_delta * alpha
        weights_1_2 -= np.outer(layer_1,layer_2_delta) * alpha

        if(np.abs(layer_2_delta) < 0.5):
            correct += 1
        total += 1
        if(i % 10 == 9):
            progress = str(i/float(len(input_dataset)))
            sys.stdout.write('\rIter:'+str(iter)\
                             +' Progress:'+progress[2:4]\
                             +'.'+progress[4:6]\
                             +'% Training Accuracy:'\
                             + str(correct/float(total)) + '%')
    print()
    
correct,total = (0,0)
for i in range(len(input_dataset)-1000,len(input_dataset)):

    x = input_dataset[i]
    y = target_dataset[i]

    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
    
    if(np.abs(layer_2 - y) < 0.5):
        correct += 1
    total += 1
print("Test Accuracy:" + str(correct / float(total)))




Iter:0 Progress:95.99% Training Accuracy:0.8351666666666666%
Iter:1 Progress:95.99% Training Accuracy:0.867875%
Test Accuracy:0.842


Let's get the vector for a word

In [18]:
target = "dentist"
target_index = word2index[target]
weights_0_1[target_index]

array([-0.12085999,  0.01579756, -0.04402984,  0.06037102,  0.03065472,
        0.0491681 ,  0.07140645, -0.01162336, -0.0643415 , -0.0144538 ,
       -0.06453222,  0.03106599,  0.01021737, -0.01537564,  0.08745243,
        0.0126589 ,  0.0092462 ,  0.01766605,  0.07138597, -0.00122123,
       -0.01193891, -0.07418969,  0.00855573,  0.05059126, -0.05761657,
       -0.07349556,  0.02761527,  0.04599354, -0.02760981,  0.08398263,
        0.06496606, -0.01751162, -0.02660331, -0.05589735, -0.06919278,
       -0.00436292, -0.04916277,  0.07664812,  0.08957953, -0.00248198,
       -0.04182991, -0.10662401,  0.09408855,  0.0236592 ,  0.04201559,
       -0.08715717, -0.11627798, -0.00431359,  0.08973731,  0.12633093,
       -0.02567942, -0.06584029,  0.00762956,  0.02060376, -0.01176154,
        0.05005198,  0.0455224 , -0.06958399,  0.00831156, -0.10808867,
       -0.02163461, -0.04908731, -0.10085263, -0.03932653, -0.07498955,
       -0.06300718,  0.0908772 , -0.01080154,  0.05342929, -0.01

# Word2vec

In [19]:
!wget http://mattmahoney.net/dc/text8.zip -O text8.zip

--2020-01-25 21:58:59--  http://mattmahoney.net/dc/text8.zip
Resolving mattmahoney.net (mattmahoney.net)... 67.195.197.75
Connecting to mattmahoney.net (mattmahoney.net)|67.195.197.75|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31344016 (30M) [application/zip]
Saving to: ‘text8.zip’

text8.zip           100%[===================>]  29.89M   712KB/s    in 44s     

2020-01-25 21:59:43 (700 KB/s) - ‘text8.zip’ saved [31344016/31344016]



In [20]:
!unzip text8.zip

Archive:  text8.zip
  inflating: text8                   


In [21]:
!pip install gensim  

In [0]:
from gensim.models import word2vec

In [23]:
# https://github.com/dav/word2vec/blob/master/scripts/create-text8-vector-data.sh#L16
model = word2vec.Word2Vec(corpus_file='text8',size=100, window=5, min_count=1, workers=4)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Let's get the vector for a word

In [24]:
vector = model.wv['computer']
print(vector)

[ 0.01222711 -0.04406689  0.02292307 -0.14559111 -0.01382657 -0.09186368
  0.04178589  0.15507385 -0.00927388 -0.00023368 -0.09715788  0.04534705
 -0.01486373  0.03722399 -0.09487018  0.07684994  0.01722169 -0.02226913
  0.0888881   0.02366327  0.07280878  0.01984839 -0.0197267   0.03881456
 -0.04799425 -0.00572116 -0.02386614  0.07351526 -0.05578825  0.08165653
 -0.00060021 -0.01289876  0.05012068  0.04070031 -0.02558472 -0.037973
  0.03656049 -0.07205242 -0.04091285 -0.0300201  -0.02913104  0.07056856
 -0.00344718  0.01811974  0.01794733  0.05565326  0.0848998  -0.01165694
  0.03508577  0.05081242  0.05895657  0.00123969 -0.06455091 -0.10328375
 -0.03349537  0.0146216   0.00760843  0.0468684   0.00093274 -0.10765529
  0.09265824 -0.12973243  0.0871974   0.06508666  0.01230119 -0.05641935
  0.05897065  0.0489506  -0.00781445 -0.13862428  0.08007901  0.05062443
 -0.00979697  0.00113295 -0.00287871  0.00411141 -0.04734411  0.03016746
 -0.01477121 -0.13873401  0.00351642  0.06382609 -0.0

## Get the similarity scores of the three embeddings

In [0]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
pd.options.mode.chained_assignment = None  # default='warn'
for num in randints:
  word1 = simlex_rand.loc[simlex_rand.index == num]['word1'].values[0]
  word2 = simlex_rand.loc[simlex_rand.index == num]['word2'].values[0]
  print('Embedding this pair from Simlex_999: "{}" and "{}"'.format(word1, word2))
  print('Running universal sentence encoder on "{}"'.format(word1))
  messages = [word1]
  # Reduce logging output.
  tf.logging.set_verbosity(tf.logging.ERROR)

  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(messages))

    for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
      print("Message: {}".format(messages[i]))
      print("Embedding size: {}".format(len(message_embedding)))
      message_embedding_snippet = ", ".join(
          (str(x) for x in message_embedding[:3]))
      print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

  vector1 = message_embedding
  vector1 = np.asarray(vector1)
  vector1 = vector1.reshape(1,512)

  print('Running universal sentence encoder on "{}"'.format(word2))
  messages = [word2]

  # Reduce logging output.
  tf.logging.set_verbosity(tf.logging.ERROR)

  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))

  vector2 = message_embedding
  vector2 = np.asarray(vector2)
  vector2 = vector2.reshape(1,512)

  cos_sim = cosine_similarity(vector1, vector2)
  print('Cosine similarity from Universal sentence encoder for {} and {} = {}'.format(word1,word2,cos_sim) )
  simlex_rand.loc[simlex_rand.index == num,'USE Cosine Sim'] = cos_sim

Embedding this pair from Simlex_999: "street" and "alley"
Running universal sentence encoder on "street"
Message: street
Embedding size: 512
Embedding: [-0.00882811564952135, -0.03262665122747421, 0.043606288731098175, ...]

Running universal sentence encoder on "alley"
Message: alley
Embedding size: 512
Cosine similarity from Universal sentence encoder for street and alley = [[0.70212521]]
Embedding this pair from Simlex_999: "car" and "carriage"
Running universal sentence encoder on "car"
Message: car
Embedding size: 512
Embedding: [-0.042257197201251984, 0.02706453762948513, -0.008567669428884983, ...]

Running universal sentence encoder on "carriage"
Message: carriage
Embedding size: 512
Cosine similarity from Universal sentence encoder for car and carriage = [[0.63279055]]
Embedding this pair from Simlex_999: "accept" and "deliver"
Running universal sentence encoder on "accept"
Message: accept
Embedding size: 512
Embedding: [0.03354945033788681, 0.006094226613640785, -0.0347519442

Embedding method 2

In [27]:
for num in randints:
  word1 = simlex_rand.loc[simlex_rand.index == num]['word1'].values[0]
  word2 = simlex_rand.loc[simlex_rand.index == num]['word2'].values[0]
  print('running word2index on the pair {} and {}'.format(word1,word2))
  target_index1 = word2index[word1]
  target_index2 = word2index[word2]
  vector1 = weights_0_1[target_index1]
  vector1 = vector1.reshape(1,100)
  vector2 = weights_0_1[target_index2]
  vector2 = vector2.reshape(1,100)
  cos_sim = cosine_similarity(vector1, vector2)
  print('Cosine similarity from word2index for {} and {} = {}'.format(word1,word2,cos_sim))
  simlex_rand.loc[simlex_rand.index == num,'word2index Cosine Sim'] = cos_sim


running word2index on the pair street and alley
Cosine similarity from word2index for street and alley = [[-0.0625077]]
running word2index on the pair car and carriage
Cosine similarity from word2index for car and carriage = [[-0.11082377]]
running word2index on the pair accept and deliver
Cosine similarity from word2index for accept and deliver = [[0.07767804]]


Embedding method word2vec



In [28]:
for num in randints:
  word1 = simlex_rand.loc[simlex_rand.index == num]['word1'].values[0]
  word2 = simlex_rand.loc[simlex_rand.index == num]['word2'].values[0]
  print('running word2vec on the pair {} and {}'.format(word1,word2))
  vector1 = model.wv[word1]
  vector1 = vector1.reshape(1,100)
  vector2 = model.wv[word2]
  vector2 = vector2.reshape(1,100)
  cos_sim = cosine_similarity(vector1, vector2)
  print('Cosine similarity from word2vec for {} and {} = {}'.format(word1,word2,cos_sim))
  simlex_rand.loc[simlex_rand.index == num,'word2vec Cosine Sim'] = cos_sim



running word2vec on the pair street and alley
Cosine similarity from word2vec for street and alley = [[0.92063576]]
running word2vec on the pair car and carriage
Cosine similarity from word2vec for car and carriage = [[-0.07631987]]
running word2vec on the pair accept and deliver
Cosine similarity from word2vec for accept and deliver = [[-0.00387929]]


In [29]:
simlex_rand_out = simlex_rand[['word1','word2','SimLex999', 'USE Cosine Sim',
       'word2index Cosine Sim', 'word2vec Cosine Sim']]
simlex_rand_out.columns = ['word1','word2','SimLex999', 'USE',
       'CBOW', 'word2vec']
simlex_rand_out = round(simlex_rand_out,4)
print('The cosine similarities from the 3 word embedding methods can be seen below.\nThe similarity scores are not close to the Simlex999 scores')
print(simlex_rand_out)

The cosine similarities from the 3 word embedding methods can be seen below.
The similarity scores are not close to the Simlex999 scores
      word1     word2  SimLex999     USE    CBOW  word2vec
178  street     alley       5.48  0.7021 -0.0625    0.9206
498     car  carriage       5.13  0.6328 -0.1108   -0.0763
971  accept   deliver       1.58  0.4180  0.0777   -0.0039


# Part 2, generate three .txt files with similarity scores of all words ending in 4, using my three embedded models.

In [53]:
## Unique id is jwb74, but python starts on zero, so I need to minus 1 to get the right words
unique_id = 4 
index = np.arange(4-1,1000,10)
simlex_out = simlex.iloc[index,:].reset_index(drop=True)[['word1','word2','SimLex999']]
simlex_out.columns = ['word1','word2','Sim_lex_Similarity']
simlex_out

,word1,word2,Sim_lex_Similarity
0,happy,cheerful,9.55
1,bad,terrible,7.78
2,dumb,foolish,6.67
3,weird,normal,0.72
4,nice,cruel,0.67
...,...,...,...
95,try,think,2.62
96,leave,appear,0.97
97,put,hang,3.00
98,say,verify,4.90


## Embedding method 1

In [54]:
import copy
simlex_out_USE = copy.deepcopy(simlex_out)
for num in np.arange(0,100,1):
  word1 = simlex_out.loc[simlex_out.index == num]['word1'].values[0]
  word2 = simlex_out.loc[simlex_out.index == num]['word2'].values[0]
  try:
    print('Embedding this pair from Simlex_999: "{}" and "{}"'.format(word1, word2))
    print('Running universal sentence encoder on "{}"'.format(word1))
    messages = [word1]
    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)

    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(messages))

      for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
        print("Message: {}".format(messages[i]))
        print("Embedding size: {}".format(len(message_embedding)))
        message_embedding_snippet = ", ".join(
            (str(x) for x in message_embedding[:3]))
        print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

    vector1 = message_embedding
    vector1 = np.asarray(vector1)
    vector1 = vector1.reshape(1,512)

    print('Running universal sentence encoder on "{}"'.format(word2))
    messages = [word2]

    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)

    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(messages))

    for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
      print("Message: {}".format(messages[i]))
      print("Embedding size: {}".format(len(message_embedding)))
      message_embedding_snippet = ", ".join(
          (str(x) for x in message_embedding[:3]))

    vector2 = message_embedding
    vector2 = np.asarray(vector2)
    vector2 = vector2.reshape(1,512)

    cos_sim = cosine_similarity(vector1, vector2)
    print('Cosine similarity from Universal sentence encoder for {} and {} = {}'.format(word1,word2,cos_sim) )
    simlex_out_USE.loc[simlex_out_USE.index == num,'your_Similarity'] = cos_sim
  except:
    print('skipping the pair {} and {}'.format(word1,word2))


Embedding this pair from Simlex_999: "weird" and "normal"
Running universal sentence encoder on "weird"
Message: weird
Embedding size: 512
Embedding: [-0.015904473140835762, -0.06259489059448242, -0.04136950895190239, ...]

Running universal sentence encoder on "normal"
Message: normal
Embedding size: 512
Cosine similarity from Universal sentence encoder for weird and normal = [[0.59703013]]
Embedding this pair from Simlex_999: "dog" and "cat"
Running universal sentence encoder on "dog"
Message: dog
Embedding size: 512
Embedding: [-0.0027078501880168915, -0.016308600082993507, -0.033069927245378494, ...]

Running universal sentence encoder on "cat"
Message: cat
Embedding size: 512
Cosine similarity from Universal sentence encoder for dog and cat = [[0.71145336]]
Embedding this pair from Simlex_999: "bed" and "blanket"
Running universal sentence encoder on "bed"
Message: bed
Embedding size: 512
Embedding: [0.014972981996834278, 0.04511445015668869, -0.004066985100507736, ...]

Running u

IndexError: ignored

## Embedding method 2

In [69]:
simlex_out_CBOW = copy.deepcopy(simlex_out)
for num in np.arange(0,100,1):
  word1 = simlex_out.loc[simlex_out.index == num]['word1'].values[0]
  word2 = simlex_out.loc[simlex_out.index == num]['word2'].values[0]
  #print('running word2index on the pair {} and {}'.format(word1,word2))
  try:
    target_index1 = word2index[word1]
    target_index2 = word2index[word2]
    vector1 = weights_0_1[target_index1]
    vector1 = vector1.reshape(1,100)
    vector2 = weights_0_1[target_index2]
    vector2 = vector2.reshape(1,100)
    cos_sim = cosine_similarity(vector1, vector2)
    #print('Cosine similarity from word2index for {} and {} = {}'.format(word1,word2,cos_sim))
    simlex_out_CBOW.loc[simlex_out_CBOW.index == num,'your_Similarity'] = cos_sim
  except:
    #print('skipping the pair {} and {}'.format(word1,word2))


running word2index on the pair happy and cheerful
Cosine similarity from word2index for happy and cheerful = [[-0.2144941]]
running word2index on the pair bad and terrible
Cosine similarity from word2index for bad and terrible = [[0.88770686]]
running word2index on the pair dumb and foolish
Cosine similarity from word2index for dumb and foolish = [[0.31393751]]
running word2index on the pair weird and normal
Cosine similarity from word2index for weird and normal = [[0.53052192]]
running word2index on the pair nice and cruel
Cosine similarity from word2index for nice and cruel = [[-0.14522757]]
running word2index on the pair bad and terrific
Cosine similarity from word2index for bad and terrific = [[-0.72073495]]
running word2index on the pair harsh and cruel
Cosine similarity from word2index for harsh and cruel = [[-0.13398465]]
running word2index on the pair bad and guilty
Cosine similarity from word2index for bad and guilty = [[-0.216249]]
running word2index on the pair exotic and ra

IndexError: ignored

In [70]:
simlex_out_word2vec = copy.deepcopy(simlex_out)
for num in np.arange(0,100,1):
  word1 = simlex_out.loc[simlex_out.index == num]['word1'].values[0]
  word2 = simlex_out.loc[simlex_out.index == num]['word2'].values[0]
  try:
    #print('running word2vec on the pair {} and {}'.format(word1,word2))
    vector1 = model.wv[word1]
    vector1 = vector1.reshape(1,100)
    vector2 = model.wv[word2]
    vector2 = vector2.reshape(1,100)
    cos_sim = cosine_similarity(vector1, vector2)
    print('Cosine similarity from word2vec for {} and {} = {}'.format(word1,word2,cos_sim))
    simlex_out_word2vec.loc[simlex_out_word2vec.index == num,'your_Similarity'] = cos_sim
  except:
    #print('skipping the pair {} and {}'.format(word1,word2))



running word2vec on the pair happy and cheerful
Cosine similarity from word2vec for happy and cheerful = [[0.00732558]]
running word2vec on the pair bad and terrible
Cosine similarity from word2vec for bad and terrible = [[0.017528]]
running word2vec on the pair dumb and foolish
Cosine similarity from word2vec for dumb and foolish = [[-0.0579617]]
running word2vec on the pair weird and normal
Cosine similarity from word2vec for weird and normal = [[0.11038039]]
running word2vec on the pair nice and cruel
Cosine similarity from word2vec for nice and cruel = [[-0.20907296]]
running word2vec on the pair bad and terrific
Cosine similarity from word2vec for bad and terrific = [[-0.07257015]]
running word2vec on the pair harsh and cruel
Cosine similarity from word2vec for harsh and cruel = [[-0.03470258]]
running word2vec on the pair bad and guilty
Cosine similarity from word2vec for bad and guilty = [[0.89570284]]
running word2vec on the pair exotic and rare
Cosine similarity from word2vec 

In [0]:
from google.colab import files
simlex_out_CBOW.to_csv('CBOW.csv', sep='\t')
simlex_out_word2vec.to_csv('word2vec.csv',sep='\t')
files.download('CBOW.csv')
files.download('word2vec.csv')